In [252]:
from deap import base, creator, tools
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)


全局参数

In [253]:
TMAX = 150
K = 10
P = 3

D_total = 1e8
Cap_rocket = 150     # 可取均值 or 后续情景分析
Cap_lift = 179000
L_max = 800

Cost_rocket = 500000
Cost_lift = 100000

#


个体初始化

In [254]:
toolbox = base.Toolbox()

# 火箭发射次数（整数）
#toolbox.register("attr_R", random.randint, 0, L_max)
toolbox.register("attr_R", lambda: random.randint(int(L_max*0.8), L_max))
# 电梯运输量（连续）
#toolbox.register("attr_E", random.uniform, 0, Cap_lift)
toolbox.register("attr_E", lambda: random.uniform(Cap_lift*0.8, Cap_lift))
def init_individual():
    ind = []

    # R[k,t]
    for _ in range(K * TMAX):
        ind.append(toolbox.attr_R())

    # E[p,t]
    for _ in range(P * TMAX):
        ind.append(toolbox.attr_E())

    return creator.Individual(ind)

toolbox.register("individual", init_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


目标函数及遗传算子##暂时弃用##

In [ ]:
def evaluate_raw(ind):
    R = np.array(ind[:K*TMAX]).reshape(K, TMAX)
    E = np.array(ind[K*TMAX:]).reshape(P, TMAX)

    # ---- 成本 ----
    Z_C = (
        Cost_rocket * np.sum(R) +
        Cost_lift * np.sum(E)
    )

    # ---- 工期 ----
    transported = 0
    Z_T = TMAX

    for t in range(TMAX):
        transported += (
            Cap_rocket * np.sum(R[:, t]) +
            np.sum(E[:, t])
        )
        if transported >= D_total:
            Z_T = t + 1
            break

    # ---- 罚函数（硬约束）----
    penalty = 0
    if transported < D_total:
        penalty = max(0, (D_total - transported)/D_total) * 1e6

    return Z_C + penalty, Z_T + penalty


#遗传算子
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=Cap_lift*0.3, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

def mutate_bound(ind):
    for i in range(len(ind)):
        if i < K*TMAX:  # 火箭次数
            ind[i] = max(0, min(L_max, ind[i]))
        else:           # 电梯运输量
            ind[i] = max(0, min(Cap_lift, ind[i]))
    return ind,


每一代种群的标准差归一化，且使用罚函数

In [241]:

def assign_fitness(pop, w1=0.5, w2=0.5):
    """
    为种群每个个体分配 fitness，使用加权求和法。
    对 ZC 和 ZT 进行种群内归一化（标准差标准化）。
    
    参数:
        pop: list of Individual
        w1, w2: 加权系数
    """
    ZC_list = []
    ZT_list = []
    
    # ---- 第一步：先计算每个个体的原始 ZC, ZT ----
    for ind in pop:
        # 解码染色体
        R = np.array(ind[:K*TMAX]).reshape(K, TMAX)
        E = np.array(ind[K*TMAX:]).reshape(P, TMAX)

        # 成本
        ZC = Cost_rocket * np.sum(R) + Cost_lift * np.sum(E)
        
        # 工期
        transported = 0
        ZT = TMAX
        for t in range(TMAX):
            R_t = np.sum(R[:, t]) * Cap_rocket
            E_t = np.sum(E[:, t])
            
            # 防止负数运输
            R_t = max(0, R_t)
            E_t = max(0, E_t)
            
            transported += R_t + E_t
            
            if transported >= D_total:
                ZT = t + 1
                break


        
        # 硬约束罚函数
        penalty = 0
        if transported < D_total:
            penalty += 1e99  # 仅加到 fitness，不改变 ZC, ZT
        
        # 保存到个体
        ind.ZC = ZC
        ind.ZT = ZT
        ind.penalty = penalty

        # 收集到列表中用于归一化
        ZC_list.append(ZC)
        ZT_list.append(ZT)
    
    # ---- 第二步：种群内归一化 ----
    std_C = np.std(ZC_list) + 1e-6
    std_T = np.std(ZT_list) + 1e-6
    
    for ind in pop:
        fitness_val = w1 * ind.ZC / std_C + w2 * ind.ZT / std_T + ind.penalty
        ind.fitness.values = (fitness_val,)


#遗传算子
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=Cap_lift*0.3, indpb=0.1)
# 可以加一个上下界限制，避免负数
def mutate_bound(ind, mu=0, sigma=Cap_lift*0.3, indpb=0.1):
    for i in range(len(ind)):
        if np.random.random() < indpb:
            ind[i] += random.gauss(mu, sigma)
            # 修剪上下界
            if i < K*TMAX:
                ind[i] = max(0, min(L_max, int(round(ind[i]))))
            else:
                ind[i] = max(0, min(Cap_lift, ind[i]))
    return ind,
toolbox.register("mutate", mutate_bound)
toolbox.register("select", tools.selTournament, tournsize=3)



若w1与w2不定的演化循环簇

In [214]:
def sweep_weights(toolbox, pop_size=50, NGEN=40, w1_values=50):
    """
    对权重 w1 从0到1取等距值，运行GA，返回每个权重下每一代的最优解。
    
    参数：
        toolbox: DEAP 注册好的 toolbox（含 individual, population, mate, mutate, select）
        pop_size: 种群大小
        NGEN: 迭代代数
        w1_values: 权重 w1 的等分数（整数或列表）
        
    返回：
        df: pandas DataFrame，列 = ['w1','w2','ZC','ZT','Z']
    """
    
    # 如果 w1_values 是整数，就生成等距值
    if isinstance(w1_values, int):
        w1_list = np.linspace(0, 1, w1_values)
    else:
        w1_list = np.array(w1_values)
    
    records = []  # 用来存储每个权重下每代最优解
    
    for w1 in w1_list:
        w2 = 1 - w1
        
        # 初始化种群
        pop = toolbox.population(n=pop_size)
        
        # 第一次计算 fitness
        assign_fitness(pop, w1=w1, w2=w2)  # 需要你的 assign_fitness 函数支持 w1, w2 参数
        
        for gen in range(NGEN):
            # 选择
            offspring = toolbox.select(pop, len(pop))
            offspring = list(map(toolbox.clone, offspring))
            
            # 交叉
            for c1, c2 in zip(offspring[::2], offspring[1::2]):
                if np.random.random() < 0.7:
                    toolbox.mate(c1, c2)
                    del c1.fitness.values, c2.fitness.values
            
            # 变异
            for ind in offspring:
                if np.random.random() < 0.2:
                    toolbox.mutate(ind)
                    del ind.fitness.values
            
            # 重新计算 fitness
            assign_fitness(offspring, w1=w1, w2=w2)
            pop[:] = offspring
            
            # 找每代最优解
            best = tools.selBest(pop, 1)[0]
            
        # 保存到 records
        records.append({
            'w1': w1,
            'w2': w2,
            'ZC': best.ZC,
            'ZT': best.ZT,
            'Z': best.fitness.values[0],
        })
    
    # 转换为 DataFrame
    df = pd.DataFrame(records)
    return df

In [222]:
df_results = sweep_weights(toolbox, pop_size=50, NGEN=50, w1_values=50)

df_results.to_csv(r"D:\USELESS\数据分析学习\数学建模学习\2026美赛\B\prob01_result1.csv")

KeyboardInterrupt: 

main函数（主演化循环）

In [248]:
def main():
    pop = toolbox.population(n=50)
    NGEN = 180
    CXPB = 0.7
    MUTPB = 0.2

    assign_fitness(pop)

    for gen in range(NGEN):
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        for c1, c2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(c1, c2)
                del c1.fitness.values, c2.fitness.values

        for ind in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(ind)
                mutate_bound(ind)
                del ind.fitness.values

        assign_fitness(offspring)
        pop[:] = offspring

        best = tools.selBest(pop, 1)[0]
        print(f"Gen {gen}: Z={best.fitness.values[0]:.3e}, "
              f"ZC={best.ZC:.3e}, ZT={best.ZT}")


    return tools.selBest(pop, 1)[0]


In [251]:
best_ind = main()


R = np.array(best_ind[:K*TMAX]).reshape(K, TMAX)
E = np.array(best_ind[K*TMAX:]).reshape(P, TMAX)

ROCKET = pd.DataFrame(R)
ELEVATOR = pd.DataFrame(E)

ROCKET.to_excel((r"D:\USELESS\数据分析学习\数学建模学习\2026美赛\B\ROCKET1.xlsx"))
ELEVATOR.to_excel((r"D:\USELESS\数据分析学习\数学建模学习\2026美赛\B\ELEVATOR1.xlsx"))
print("最终总运输量 =", np.sum(R)*Cap_rocket + np.sum(E))
print('火箭运输量 = ', np.sum(R)*Cap_rocket)
print('太空电梯运输量 = ', np.sum(E))

Gen 0: Z=8.496e+01, ZC=7.747e+12, ZT=64
Gen 1: Z=6.292e+01, ZC=7.559e+12, ZT=68
Gen 2: Z=5.331e+01, ZC=7.592e+12, ZT=65
Gen 3: Z=4.903e+01, ZC=7.592e+12, ZT=65
Gen 4: Z=4.322e+01, ZC=7.445e+12, ZT=65
Gen 5: Z=5.271e+01, ZC=7.445e+12, ZT=65
Gen 6: Z=5.417e+01, ZC=7.445e+12, ZT=65
Gen 7: Z=5.070e+01, ZC=7.395e+12, ZT=66
Gen 8: Z=4.537e+01, ZC=7.384e+12, ZT=66
Gen 9: Z=5.080e+01, ZC=7.208e+12, ZT=68
Gen 10: Z=5.059e+01, ZC=7.172e+12, ZT=68
Gen 11: Z=5.273e+01, ZC=7.073e+12, ZT=70
Gen 12: Z=6.128e+01, ZC=7.023e+12, ZT=70
Gen 13: Z=4.334e+01, ZC=6.968e+12, ZT=71
Gen 14: Z=4.267e+01, ZC=6.968e+12, ZT=71
Gen 15: Z=4.577e+01, ZC=6.903e+12, ZT=71
Gen 16: Z=5.366e+01, ZC=6.915e+12, ZT=70
Gen 17: Z=6.990e+01, ZC=6.824e+12, ZT=72
Gen 18: Z=5.294e+01, ZC=6.923e+12, ZT=69
Gen 19: Z=5.711e+01, ZC=6.762e+12, ZT=72
Gen 20: Z=5.292e+01, ZC=6.666e+12, ZT=72
Gen 21: Z=4.913e+01, ZC=6.514e+12, ZT=75
Gen 22: Z=5.122e+01, ZC=6.522e+12, ZT=74
Gen 23: Z=5.379e+01, ZC=6.522e+12, ZT=74
Gen 24: Z=5.231e+01, ZC=6.